In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create a persistent working directory in your Drive
import os
drive_dir = '/content/drive/MyDrive/colab_dnn_mode_connectivity'
os.makedirs(drive_dir, exist_ok=True)
drive_dir


In [ ]:
# Always clone into the VM runtime (fast, clean)
!git clone https://github.com/xenakistheo/dnn-mode-connectivity.git

import os
os.chdir("dnn-mode-connectivity")

# Optional: pull latest changes if the folder already existed (rare)
!git pull


In [ ]:
# learning rate   = 0.1
# momentum        = 0.9
# weight decay    = 3e-4
# epochs          = 80

# ResNets: 8, 26, 38, 65, 119.

#Remember to set the MODEL variable below
MODEL = "ResNet8"


# This seldom need to be changed
DATASET = "CIFAR10"
SEED1 = "1"
SEED2 = "2"

Use the following names for directories
- Endpoints [model_name]\_EP\_[seednr.]. E.g. ResNet80_EP_1
- Connecting path [model_name]\_CP\_[pathtype]\_[seednr.1]\_[seednr.2]. E.g. ResNet8_CP_Bezier_1_2
- Path evaluation [model_name]\_EV\_[pathtype]\_[seednr.1]\_[seednr.2]. E.g. ResNet8_EV_Bezier_1_2
- Hessian Analysis [model_name]\_HA\_[pathtype]\_[seednr.1]\_[seednr.2]. E.g. ResNet8_HA_Bezier_1_2

In [ ]:
### Train Endpoint 1
# Remember to
# 1. Set correct directory
# 2. Set correct model
!python3 train.py --dir=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED1} \
  --dataset=$DATASET --data_path=./data --transform=ResNet \
  --model=$MODEL \
  --epochs=80 --lr=0.1 --wd=3e-4 --save_freq=40 --use_test \
  --seed=1

#A100
#ResNet8 - 8min
#ResNet26 - 9min
#ResNet65 - 18min
#ResNet119 - 31min

In [ ]:
### Train Endpoint 2
# Remember to
# 1. Set correct directory
# 2. Set correct model
!python3 train.py --dir=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED2} \
        --dataset=$DATASET --data_path=./data --transform=ResNet \
        --model=$MODEL \
        --epochs=80 --lr=0.1 --wd=3e-4 --save_freq=40 --use_test \
        --seed=2

#A100
#ResNet8 - 8min
#ResNet26 - 9min
#ResNet65 - 18min

In [ ]:
### Find Connecting Path - Bezier
# Remember to
# 1. Set correct directory
# 2. Set correct models
# Set correct endpoint paths
!python3 train.py --dir=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_CP_Bezier_{SEED1}_{SEED2} \
        --dataset=$DATASET --use_test --transform=ResNet --data_path=./data \
        --model=$MODEL \
        --curve=Bezier \
        --num_bends=3 \
        --init_start=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED1}/checkpoint-80.pt \
        --init_end=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED2}/checkpoint-80.pt \
        --fix_start --fix_end --epochs=50 --lr=0.1 --wd=3e-4

#A100
#ResNet8 - 5min
#ResNet65 - 30min

In [ ]:
### Find Connecting Path - PolyChain
# Remember to
# 1. Set correct directory
# 2. Set correct models
# Set correct endpoint paths/content/drive/MyDrive/mode_connectivity_runs/PreResNet_e
!python3 train.py --dir=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_CP_PolyChain_{SEED1}_{SEED2} \
        --dataset=$DATASET --use_test --transform=ResNet --data_path=./data \
        --model=$MODEL \
        --curve=PolyChain \
        --num_bends=3 \
        --init_start=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED1}/checkpoint-80.pt \
        --init_end=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED2}/checkpoint-80.pt \
        --fix_start --fix_end --epochs=50 --lr=0.1 --wd=3e-4

#A100
#ResNet8 - 5min
#ResNet65 - 30min

In [ ]:
### Evaluate Path - Bezier
# Remember to
# 1. Set correct directory
# 2. Set correct models
# Set correct endpoint paths
!python3 eval_curve.py --dir=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EV_Bezier_{SEED1}_{SEED2} \
        --dataset=$DATASET --data_path=./data --transform=ResNet \
        --model=$MODEL \
        --wd=3e-4 \
        --curve=Bezier \
        --num_bends=3 \
        --ckpt=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_CP_Bezier_{SEED1}_{SEED2}/checkpoint-50.pt\
        --num_points=61 --use_test

#A100
#ResNet8 - 12min
#ResNet65 - 46min

In [ ]:
### Evaluate Path - PolyChain
# Remember to
# 1. Set correct directory
# 2. Set correct models
# Set correct endpoint paths
!python3 eval_curve.py --dir=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EV_PolyChain_{SEED1}_{SEED2} \
        --dataset=$DATASET --data_path=./data --transform=ResNet \
        --model=$MODEL \
        --wd=3e-4 \
        --curve=PolyChain \
        --num_bends=3 \
        --ckpt=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_CP_PolyChain_{SEED1}_{SEED2}/checkpoint-50.pt \
        --num_points=61 --use_test

#A100
#ResNet8 - 12min

In [ ]:
# ### Perform Hessian Analysis
# # Remember to
# # 1. Set correct directory
# # 2. Set correct models
# # Set correct endpoint paths

for _CURVE in ["Bezier", "PolyChain"]:
    !python3 hessian_curve_analysis.py --curve_dir=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_HA_{_CURVE}_1_2 \
            --curve_ckpt=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_CP_{_CURVE}_1_2/checkpoint-50.pt \
            --dataset={DATASET} --data_path=./data \
            --model={MODEL} \
            --curve={_CURVE} \
            --num_bends=3 --batch_size=128 --use_test

# L4 (generalization gap + hessian analysis)
# ResNet8 = 16 min + 5 min
# ResNet26 = 29 min + 14 min
# ResNet38 = 34 min + 18 min
# Resnet65 = 35 min + 45 min
# Resnet119 = 50 min + 90 min

----

In [ ]:
### Convex Combination Evaluation 
## - Measure performance along the linear path between the two endpoints
# Remember to
# 1. Set correct model
# 2. Set correct endpoint paths
# 3. Set correct output csv file name

!python convex_combo.py \
    --ckpt_a=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED1}/checkpoint-80.pt \
    --ckpt_b=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED2}/checkpoint-80.pt \
    --model=$MODEL \
    --steps=31 \
    --recompute_bn \
    --data_path=./data \
    --save_csv=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_convex.csv

In [ ]:
### Get Angle Data 
## - Currently does not save to csv in proper way
# Remember to
# 1. Set correct model
# 2. Set correct endpoint paths
# 3. Set correct output csv file name

!python param_angle.py \
    -a=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED1}/checkpoint-80.pt \
    -c=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_CP_Bezier_{SEED1}_{SEED2}/checkpoint-50.pt \
    -b=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_EP_{SEED2}/checkpoint-80.pt \
    --curve-bend-index 1 \
    --save_csv /content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_angle.csv

In [ ]:
!python3 plane.py --dir=/content/drive/MyDrive/mode_connectivity_runs/results/plane_data \
    --ckpt=/content/drive/MyDrive/mode_connectivity_runs/{MODEL}_{DATASET}_CP_PolyChain_{SEED1}_{SEED2}/checkpoint-50.pt \
    --model=$MODEL \
    --curve=Bezier \
    --num_bends=3 \
    --dataset=$DATASET \
    --data_path=./data \
    --grid_points=21 \
    --curve_points=61 \
    --batch_size=128